In [75]:
import import_ipynb
import api
import random
import string
import json

baseUrl = 'http://localhost:8080'

def main():
  test_accessibility()
  
  # accessibility
  email = random_string() + '@example.com'
  password = random_string()
  
  # user
  test_create_user(email, password)
  u = test_get_user(email)
  assert u[0] == email
  certs = testGetCertificates(email)
  assert len(certs) == 0
  
  # certificates
  test_create_certificate(email)
  certs = test_get_certificates(email)
  assert len(certs) == 1
  cert = certs[0]
  assert len(cert) == 6

  certificate_id = cert[0]
  certificate_email = cert[1]
  is_active = cert[5]
  assert certificate_email == email
  assert is_active    # certificate is active
  
  # deactivate certificate
  test_update_certificate_status(email, certificate_id, False)
  certs = test_get_certificates(email)
  is_active = certs[0][5]
  assert not is_active
  
  # activate certificate
  test_update_certificate_status(email, certificate_id, True)
  certs = test_get_certificates(email)
  is_active = certs[0][5]
  assert is_active
  
  # delete user
  test_delete_user(email)
  u = test_get_user(email)
  assert not u # user doesn't exist

  
def random_string(N=10):
  return ''.join(random.choices(string.ascii_letters + string.digits, k=N))
  
def test_accessibility():
  resp = api.make_get(baseUrl)
  assert resp.content == b'Hello, World!'
  assert resp.status_code == 200
  print("=========accessibility test passed\n")

def test_create_user(email, password):
  print('creating test user with email {} and password {}'.format(email, password))
  data = {"password": password}
  resp = api.make_post(baseUrl + '/users/' + email, data)
  assert resp.status_code == 200
  print('=========user {} successfully created\n'.format(email))
  
def test_get_user(email):
  print('getting test user with email {}'.format(email))
  resp = api.make_get(baseUrl + '/users/' + email)
  assert resp.status_code == 200
  u = json.loads(resp.content)
  print("=========get user response content successfully: {}\n".format(u))
  return u

def test_delete_user(email):
  print('deleting test user with email {}'.format(email))
  resp = api.make_delete(baseUrl + '/users/' + email)
  assert resp.status_code == 200
  print("=========delete user {} successfully\n".format(email))
  
def test_get_certificates(email):
  print('getting certificates from test user {}'.format(email))
  resp = api.make_get('/'.join([baseUrl, 'users', email, 'certificates']))
  certs = json.loads(resp.content)
  print('=========get user {}\'s certificates successfully: {}\n'.format(email, certs))
  return certs

def test_create_certificate(email):
  private_key = random_string()
  certificate = random_string()
  data = {
    "private_key": private_key,
    "certificate": certificate
  }
  print('creating certificate for test user()'.format(email))
  resp = api.make_post('/'.join([baseUrl, 'users', email, 'certificates']), data)
  assert resp.status_code == 200
  print('=========certificate created successfully\n')

def test_update_certificate_status(email, certificate_id, active):
  if active:
    print('activating certificate {} for test user {}'.format(certificate_id, email))
  else:
    print('deactivating certificate {} for test user {}'.format(certificate_id, email))
  data = {
    "active": active
  }
  resp = api.make_patch('/'.join([baseUrl, 'certificates', str(certificate_id)]), data)
  assert resp.status_code == 200
  print("=========update certificate status successfully\n")

import imp  
imp.reload(api)  
main()

importing Jupyter notebook from api.ipynb
Getting from URL: http://localhost:8080
=========accessibility test passed

creating test user with email w0hcZRiSUR@example.com and password spy6c8fNEO
Posting {'password': 'spy6c8fNEO'} to URL: http://localhost:8080/users/w0hcZRiSUR@example.com
=========user w0hcZRiSUR@example.com successfully created

getting test user with email w0hcZRiSUR@example.com
Getting from URL: http://localhost:8080/users/w0hcZRiSUR@example.com
=========get user response content successfully: ['w0hcZRiSUR@example.com', 'Mon, 06 May 2019 03:17:47 GMT']

getting certificates from test user w0hcZRiSUR@example.com
Getting from URL: http://localhost:8080/users/w0hcZRiSUR@example.com/certificates
get user w0hcZRiSUR@example.com's certificates successfully: b'[]\n'
creating certificate for test user()
Posting {'private_key': 'H7InMvsWyM', 'certificate': 'wEF9urnF7o'} to URL: http://localhost:8080/users/w0hcZRiSUR@example.com/certificates
=========certificate created succes